In [1]:
import sys
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

sys.path.append(parent_dir)

from src.utils import load_MNIST_data
from src.model import simple_model
from src.train import train_model

import torch
import torch.nn as nn


In [2]:
train_dloader, val_dloader, test_dloader = load_MNIST_data(batch_size = 64)

CNN_model = simple_model(out_channels = 6, 
                         kernel_size = 5)

loss_fcn = nn.CrossEntropyLoss(reduction='sum')
optimiser = torch.optim.SGD(params = CNN_model.parameters(), lr = 0.001, momentum = 0.9)

for epoch in range(2):
    train_loss, val_loss, val_acc = train_model(CNN_model, optimiser, loss_fcn, train_dloader, val_dloader, epoch, print_training = True)
    



100%|██████████| 26421880/26421880 [00:05<00:00, 4657068.19it/s]


Extracting ../data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 1346416.11it/s]

Extracting ../data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw




100%|██████████| 4422102/4422102 [00:00<00:00, 4454454.46it/s]


Extracting ../data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 5790366.58it/s]


Extracting ../data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



[Epoch  1] Training accuracy: 82.43%, Validation accuracy: 84.97%



[Epoch  2] Training accuracy: 86.07%, Validation accuracy: 85.61%

